In [ ]:
import os
import glob
import pandas as pd
import json
import numpy as np

def load_latest_events(basepath="concert_events"):
    """
    Find the latest JSON file saved by save_events_csv() and return a list of dicts.
    """
    pattern = f"{basepath}_*.json"
    files = glob.glob(pattern)
    if not files:
        raise FileNotFoundError(f"No files found matching {pattern}")

    latest_file = max(files, key=os.path.getmtime)

    with open(latest_file, "r", encoding="utf-8") as f:
        events = json.load(f)

    print(f"✅ Loaded {len(events)} events from {latest_file}")
    return events

loaded_events = load_latest_events("concert_events")

✅ Loaded 374 events from concert_events_20250919_005906.json


In [ ]:
# ingest.py
import meilisearch

# Convert to list of dicts for Meilisearch
events = load_latest_events("concert_events")

# Connect to Meilisearch
client = meilisearch.Client("http://localhost:7700", "tiMpun-mipvy5-tehxiw")
index = client.index("events")
index.add_documents(events, primary_key="id")

# Configure searchable/filterable/sortable fields
index.update_searchable_attributes(["band", "location", "status_kind", "status_raw"])
index.update_filterable_attributes(["location", "status_kind", "price_eur", "date", "band"])
index.update_sortable_attributes(["price_eur", "date"])

print(f"✅ Indexed {len(events)} events into Meilisearch")

✅ Loaded 374 events from concert_events_20250919_005906.json
✅ Indexed 374 events into Meilisearch


In [2]:
import meilisearch

# Convert to list of dicts for Meilisearch
#events = load_latest_events("concert_events")

# Connect to Meilisearch
client = meilisearch.Client("http://localhost:7700", "tiMpun-mipvy5-tehxiw")
index = client.index("events")

facet_fields = ["location", "status_kind", "price_eur", "date", "band"]

result = index.search(
    "sonic",
    {
        "filter": 'price_eur < 50 AND status_kind != "ausverkauft"',
        "facets": facet_fields,
        "sort": ["date:desc"],
        "limit": 1000,
    }
)

result

{'hits': [{'id': 371,
   'origin': '20.09. The Vovos *15,40 €',
   'date': '2026-09-20T00:00:00',
   'band': 'The Vovos',
   'location': 'Sonic Ballroom',
   'price_eur': 15.4,
   'status_kind': '',
   'new_location': '',
   'status_raw': '',
   'section': 'Sonic Ballroom'},
  {'id': 366,
   'origin': '19.09. Popperklopper *14,30 €',
   'date': '2026-09-19T00:00:00',
   'band': 'Popperklopper',
   'location': 'Sonic Ballroom',
   'price_eur': 14.3,
   'status_kind': '',
   'new_location': '',
   'status_raw': '',
   'section': 'Sonic Ballroom'},
  {'id': 360,
   'origin': '16.09. Lord Rochester *16,50 €',
   'date': '2026-09-16T00:00:00',
   'band': 'Lord Rochester',
   'location': 'Sonic Ballroom',
   'price_eur': 16.5,
   'status_kind': '',
   'new_location': '',
   'status_raw': '',
   'section': 'Sonic Ballroom'},
  {'id': 354,
   'origin': '13.09. The Slapstickers *17,60 €',
   'date': '2026-09-13T00:00:00',
   'band': 'The Slapstickers',
   'location': 'Sonic Ballroom',
   'price